In [1]:
# coding:utf-8

import pandas as pd
import numpy as np
import os,sys
from multiprocessing import Pool
import matplotlib.pyplot as plt

In [2]:
baseline_test = pd.read_csv('../datas/baseline_1part_test')
baseline_test.head()

,device_city,device_cnt,device_cnt_0,device_cnt_1,device_cnt_1rate,device_cnt_rate,device_cnt_scan0,device_cnt_scan0rate,device_cnt_scan1,device_cnt_sec0,...,iptype_cnt_sec1,iptype_device,iptype_diff1,iptype_diff2,iptype_diff3,iptype_from,iptype_id,iptype_time_max,iptype_time_mean,iptype_time_min
0,199,6991,1345,5646,0.807494,0.192363,6945,0.993278,46,6991,...,0,1,971374.0,1423633.0,1776467.0,1,3,8107.0,7249.666667,5775.0
1,1,3,1,2,0.500000,0.250000,3,0.750000,0,3,...,0,1,385.0,393.0,437741.0,1,1,11777.0,6608.333333,3978.0
2,0,0,0,0,0.000000,0.000000,0,0.000000,0,0,...,0,1,4931.0,4934.0,7436.0,1,1,11655.0,7514.400000,3982.0
3,1,1,0,1,0.500000,0.000000,1,0.500000,0,1,...,0,0,NaN,NaN,NaN,0,0,NaN,NaN,NaN
4,2,8,0,8,0.888889,0.000000,8,0.888889,0,8,...,0,0,NaN,NaN,NaN,0,0,NaN,NaN,NaN


In [3]:
baseline_test.shape

(87748, 204)

In [4]:
baseline_test = baseline_test.set_index('idx')
baseline_test.head()

,device_city,device_cnt,device_cnt_0,device_cnt_1,device_cnt_1rate,device_cnt_rate,device_cnt_scan0,device_cnt_scan0rate,device_cnt_scan1,device_cnt_sec0,...,iptype_cnt_sec1,iptype_device,iptype_diff1,iptype_diff2,iptype_diff3,iptype_from,iptype_id,iptype_time_max,iptype_time_mean,iptype_time_min
idx,,,,,,,,,,,,,,,,,,,,,
125112,199,6991,1345,5646,0.807494,0.192363,6945,0.993278,46,6991,...,0,1,971374.0,1423633.0,1776467.0,1,3,8107.0,7249.666667,5775.0
125113,1,3,1,2,0.500000,0.250000,3,0.750000,0,3,...,0,1,385.0,393.0,437741.0,1,1,11777.0,6608.333333,3978.0
125114,0,0,0,0,0.000000,0.000000,0,0.000000,0,0,...,0,1,4931.0,4934.0,7436.0,1,1,11655.0,7514.400000,3982.0
125115,1,1,0,1,0.500000,0.000000,1,0.500000,0,1,...,0,0,NaN,NaN,NaN,0,0,NaN,NaN,NaN
125116,2,8,0,8,0.888889,0.000000,8,0.888889,0,8,...,0,0,NaN,NaN,NaN,0,0,NaN,NaN,NaN


In [5]:
t_login = pd.read_csv('../datas/t_login.csv')
t_login_test = pd.read_csv('../datas/t_login_test.csv')

t_login = t_login[t_login['time']>='2015-06-01 00:00:00']
t_login = pd.concat([t_login,t_login_test])


t_login['timestamp_online'] = t_login['timestamp'] + t_login['timelong']
t_login['result'] = t_login['result'].map(lambda x: x == 1 and 1 or -1)

t_login = t_login.sort_values('timestamp') \
                .reset_index(drop=True)


dtt = t_login[t_login['time']>='2015-07-01 00:00:00']
dtt.head()

,log_id,timelong,device,log_from,ip,city,result,timestamp,type,id,is_scan,is_sec,time,timestamp_online
125112,1.173890e+16,4996.0,835072,1,1084528,194,-1,1.435680e+09,3,78598,False,False,2015-07-01 00:01:05,1.435685e+09
125113,3.114995e+15,9211.0,494781,1,507137,194,1,1.435680e+09,2,48119,False,False,2015-07-01 00:01:26,1.435689e+09
125114,8.936806e+15,6000.0,218435,2,136150,21,1,1.435680e+09,1,76097,False,False,2015-07-01 00:01:37,1.435686e+09
125115,3.179558e+16,9227.0,901290,1,484455,101,1,1.435680e+09,3,98066,False,False,2015-07-01 00:01:41,1.435689e+09
125116,1.856346e+16,7972.0,39412,1,1057787,83,1,1.435680e+09,1,82720,False,False,2015-07-01 00:02:28,1.435688e+09


In [6]:
dtt.shape

(87748, 14)

In [7]:
data = dtt.merge(baseline_test,left_index=True,right_index=True)
data.head()

,log_id,timelong,device,log_from,ip,city,result,timestamp,type,id,...,iptype_cnt_sec1,iptype_device,iptype_diff1,iptype_diff2,iptype_diff3,iptype_from,iptype_id,iptype_time_max,iptype_time_mean,iptype_time_min
125112,1.173890e+16,4996.0,835072,1,1084528,194,-1,1.435680e+09,3,78598,...,0,1,971374.0,1423633.0,1776467.0,1,3,8107.0,7249.666667,5775.0
125113,3.114995e+15,9211.0,494781,1,507137,194,1,1.435680e+09,2,48119,...,0,1,385.0,393.0,437741.0,1,1,11777.0,6608.333333,3978.0
125114,8.936806e+15,6000.0,218435,2,136150,21,1,1.435680e+09,1,76097,...,0,1,4931.0,4934.0,7436.0,1,1,11655.0,7514.400000,3982.0
125115,3.179558e+16,9227.0,901290,1,484455,101,1,1.435680e+09,3,98066,...,0,0,NaN,NaN,NaN,0,0,NaN,NaN,NaN
125116,1.856346e+16,7972.0,39412,1,1057787,83,1,1.435680e+09,1,82720,...,0,0,NaN,NaN,NaN,0,0,NaN,NaN,NaN


In [8]:
data.to_csv('../datas/baseline_feas',index=None)

In [12]:
dict(data.loc[125112][['id_cnt_scan1','id_device']])

{'id_cnt_scan1': 0, 'id_device': 4}